In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Question 1 -
Train a deep learning model which would classify the vegetables based on the
images provided. The dataset can be accessed from the given link.

https://www.kaggle.com/datasets/misrakahmed/vegetable-image-dataset

Note -
1. Use PyTorch as the framework for training model
2. Use Distributed Parallel Training technique to optimize training time.
3. Achieve an accuracy of at least 85% on the validation dataset.
4. Use albumentations library for image transformation
5. Use TensorBoard logging for visualizing training performance
6. Use custom modular Python scripts to train model
7. Only Jupyter notebooks will not be allowed
8. Write code comments wherever needed for understanding

In [3]:
train_dir = '../input/vegetable-image-dataset/Vegetable Images/train'
val_dir = '../input/vegetable-image-dataset/Vegetable Images/validation'
test_dir = '../input/vegetable-image-dataset/Vegetable Images/test'
classes = os.listdir(train_dir)

In [4]:
classes, train_dir, val_dir, test_dir

(['Broccoli',
  'Capsicum',
  'Bottle_Gourd',
  'Radish',
  'Tomato',
  'Brinjal',
  'Pumpkin',
  'Carrot',
  'Papaya',
  'Cabbage',
  'Bitter_Gourd',
  'Cauliflower',
  'Bean',
  'Cucumber',
  'Potato'],
 '../input/vegetable-image-dataset/Vegetable Images/train',
 '../input/vegetable-image-dataset/Vegetable Images/validation',
 '../input/vegetable-image-dataset/Vegetable Images/test')

In [43]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
from torchvision import transforms
from torchsummary import summary
from time import time

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Device is:- {device}")

Device is:- cuda


In [6]:
train_transform = transforms.Compose([
    transforms.Resize((128,128)),
    transforms.ToTensor(),
])
test_transform = transforms.Compose([
    transforms.Resize((128,128)),
    transforms.ToTensor(),
])

In [7]:
train_dataset = torchvision.datasets.ImageFolder(root=train_dir, transform=train_transform)
val_dataset = torchvision.datasets.ImageFolder(root=val_dir, transform=test_transform)
test_dataset = torchvision.datasets.ImageFolder(root=test_dir, transform=test_transform)

In [10]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=128, shuffle=True)
val_loader = torch.utils.data.DataLoader(dataset=val_dataset, batch_size=128, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=128)

In [11]:
train_dataset

Dataset ImageFolder
    Number of datapoints: 15000
    Root location: ../input/vegetable-image-dataset/Vegetable Images/train
    StandardTransform
Transform: Compose(
               Resize(size=(128, 128), interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
           )

In [23]:
class VegetableModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.seq1 = nn.Sequential(
            nn.Conv2d(3,64,3),
            nn.Conv2d(64,128,3),
            nn.MaxPool2d(3,3),
            nn.Conv2d(128,256,1),
            nn.Conv2d(256,256,3),
            nn.Conv2d(256,64,1)
        )
        self.maxpool = nn.MaxPool2d(2,2)
        self.flat = nn.Flatten(1)
        self.layer = nn.Linear(23104,15)
        self.to(device)

    def forward(self, x):
        # input = [1, 3, 224, 224]
        x = self.seq1(x)
        x = F.relu(self.maxpool(x))
        x = self.flat(x)
        x = self.layer(x)
        # output = [1, 15]
        return x

In [24]:
model = VegetableModel()
summary(model, (3,128,128))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 126, 126]           1,792
            Conv2d-2        [-1, 128, 124, 124]          73,856
         MaxPool2d-3          [-1, 128, 41, 41]               0
            Conv2d-4          [-1, 256, 41, 41]          33,024
            Conv2d-5          [-1, 256, 39, 39]         590,080
            Conv2d-6           [-1, 64, 39, 39]          16,448
         MaxPool2d-7           [-1, 64, 19, 19]               0
           Flatten-8                [-1, 23104]               0
            Linear-9                   [-1, 15]         346,575
Total params: 1,061,775
Trainable params: 1,061,775
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.19
Forward/backward pass size (MB): 31.76
Params size (MB): 4.05
Estimated Total Size (MB): 36.00
-------------------------------------

In [33]:
loss_fun = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.001
    maximize: False
    weight_decay: 0
)

In [44]:
total_batch = len(train_loader)
val_batch = len(val_loader)

print(f"total_batch {total_batch}, val_batch {val_batch}")

epochs = 10
loss_list = []
start_time = time()
for epoch in range(epochs):
    epoch_start_time = time()
    total_loss = 0
    for i, data in enumerate(train_loader):
        image, label = data
        image = image.to(device) #input
        label = label.to(device) #output

        optimizer.zero_grad()
        y_pred = model.forward(image) #Forward pass
        loss = loss_fun(y_pred, label) #loss
        total_loss += loss #loss calculate

        loss.backward() # baxkward pass
        optimizer.step() # Optimse Loss

        if (i + 1) % 23 == 0:
            print(f'Epoch [{epoch + 1}/{epochs}], Step [{i + 1}/{len(train_loader)}], Loss: {loss.item():.4f}')
    epoch_end_time = time()
    print(f"Epoch {epoch + 1} time: {(epoch_end_time - epoch_start_time):.2f}s")
    

    loss_list.append(total_loss / total_batch)
    print(f'Epochs: {epoch + 1}, Loss: {total_loss / total_batch}')

    # Validation
    with torch.no_grad():
        val_loss = 0
        correct, total = 0,0
        for i, data in enumerate(val_loader):
            val_image, val_label = data
            val_image = val_image.to(device)
            val_label = val_label.to(device)

            val_out = model.forward(val_image)
            v_loss = loss_fun(val_out, val_label)
            val_loss += v_loss
            _, predicted = torch.max(val_out.data, 1)
            total += val_label.size(0)
            correct += (predicted == val_label).sum().item()
        
        accuracy = 100 * correct / total
        print(f'Accuracy on the test images: {accuracy:.2f}%, validation_loss: {val_loss / val_batch}')
        print("-" * 50)

end_time = time()
print(f"Training time: {(end_time - start_time):.2f}s")

total_batch 118, val_batch 24
Epoch [1/10], Step [23/118], Loss: 0.0890
Epoch [1/10], Step [46/118], Loss: 0.3006
Epoch [1/10], Step [69/118], Loss: 0.0516
Epoch [1/10], Step [92/118], Loss: 0.5166
Epoch [1/10], Step [115/118], Loss: 0.1766
Epoch 1 time: 51.62s
Epochs: 1, Loss: 0.20387324690818787
Accuracy on the test images: 82.60%, validation_loss: 1.0905121564865112
--------------------------------------------------
Epoch [2/10], Step [23/118], Loss: 0.1754
Epoch [2/10], Step [46/118], Loss: 0.1186
Epoch [2/10], Step [69/118], Loss: 0.0963
Epoch [2/10], Step [92/118], Loss: 0.0037
Epoch [2/10], Step [115/118], Loss: 0.0255
Epoch 2 time: 50.61s
Epochs: 2, Loss: 0.13029228150844574
Accuracy on the test images: 92.27%, validation_loss: 0.48024511337280273
--------------------------------------------------
Epoch [3/10], Step [23/118], Loss: 0.0125
Epoch [3/10], Step [46/118], Loss: 0.0638
Epoch [3/10], Step [69/118], Loss: 0.0199
Epoch [3/10], Step [92/118], Loss: 0.0255
Epoch [3/10], S